In [58]:
from selenium import webdriver
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

In [59]:
sys.dont_write_bytecode = True
sys.path.append('../')
from tools.maneger import DataManager

In [60]:
corpus_root = "../corpus"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"

corpus_name = "What.json"
print(data_path)

../corpus/question/


In [61]:
dataM = DataManager(data_path)

In [76]:
class Crawler:
    def __init__(self, ) -> None:
        self.YN_stop_list = "Is Do Did Can Have".split()
        self.WH_stop_list = "What When Where Who What Why How".split()
        self.please_stop_list = ["please", "Would you", "Could you"]
        self.plain_stop_list = "I She He They We You".split()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def crawl(self, mode, num):
        self.mode = mode
        self.num = num
        if mode == "YN":
            self.stop_list = self.YN_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode == "WH":
            self.stop_list = self.WH_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode =="please":
            self.stop_list = self.please_stop_list
            self.essential = "?"
            self._crawl_ENG()
        elif mode == "plain":
            self.stop_list = self.plain_stop_list
            self.essential = "."
            self._crawl_ENG()
        
    
    def _in(self, search, target):
        """
        search : list or str
        target : str
        """
        result = True
        if isinstance(search, list):
            for s in search:
                result = result and s in target
            return result
        elif isinstance(search, str):
            return search in target
        else:
            print("list or str is expected, but not")
            return False
    
    def _extract_stop_type(self, target):
        if self.essential not in target:
            return None
        for stop in self.stop_list:
            if stop in target:
                return stop
        return None
    
    def _extract_label(self, text):
        if self.mode == "YN":
            return ["YN", self._extract_stop_type(text)]
        # 後で
        elif self.mode == "WH":
            return ["WH", self._extract_stop_type(text)]
        elif self.mode == "plain":
            return ["plain"]
        else:
            return ["please"]
    
    def _go_conv_page(self):
        element = self.driver.find_element_by_class_name("condition").find_elements_by_tag_name("a")
        element[1].click()
        
    def _crawl_ENG(self):
        self.data = []
        self.label = []

        self.driver.get("https://gogakuru.com/english/phrase/genre/index.html")
        
        self._go_conv_page()

        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        while 1:
            cells = soup.find_all("td", class_="summary")
            for cell in cells:
                engs = cell.find_all("span", class_="font-en")
                text_ = [eng.get_text() for eng in engs]
                text = text_[0]
                # print(text)
                # 大文字で含まれるならば
                stop = self._extract_stop_type(text)
                if  stop:
                    jp = cell.find("dd", class_="jp").get_text()
                    self.data.append(jp)
                    self.label.append( self._extract_label(text) )
                    print("stop:{0}, jp:{1}".format(stop, jp))

            # 一度打ち切り
            if len(self.data) >= self.num:
                break
        
            # 次のurl
            try:
                next_t = self.driver.find_element_by_class_name("right").find_element_by_tag_name('a')
                next_t.click()
            except:
                break
            html = self.driver.page_source.encode('utf-8')
            soup = BeautifulSoup(html, 'lxml')
        
        

    def make_corpus(self):
        self.corpus = {}
        self.corpus[self.mode] = []
        for t, l in zip(self.data, self.label):
            one_data = {"data":t, "label":l}
            self.corpus[self.mode].append(one_data)
    
    def write(self, filename):
        with open(filename, "w") as f:
            json.dump(self.corpus, f,  ensure_ascii=False, indent=4)
    
    def quit(self):
        self.driver.quit() 



In [82]:
corpus_root = "../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../corpus/question/'

In [83]:
crawler = Crawler()
mode = "WH"

corpus_name = "{0}.json".format(mode)

crawler.connect("192.168.1.224")

In [84]:
crawler.crawl(mode, 150)

stop:How, jp:締め切りまで何日ある？
stop:How, jp:この乗車時間はどれくらいだっけ？
stop:Why, jp:どうして遅くなったの？
stop:How, jp:どうやって食べるの？
stop:What, jp:何を買うつもり？
stop:Who, jp:彼に取材したい人は？
stop:Where, jp:どこから始めればいいの？
stop:Who, jp:これはだれのカバンだろう？
stop:Where, jp:そのチームはどこで練習しているのですか？
stop:Where, jp:どこへ行ったんだろう？
stop:How, jp:写真は何枚あるの？
stop:What, jp:いつも何時に家に帰っているの？
stop:When, jp:次の試合はいつですか？
stop:What, jp:それって何？
stop:When, jp:モニカはいつ、アメリカでダンスを勉強することになっているの？
stop:What, jp:モニカのことどう思う？
stop:How, jp:週末はどうだった？
stop:What, jp:ぼくの考え、どう思う？
stop:What, jp:何時に始めたの？
stop:Where, jp:先生のネックレスはどこで手に入れたんですか？
stop:When, jp:いつお店に行ったの？
stop:What, jp:きのうの夜、何をしたの？
stop:Who, jp:ココナッツを中に入れたのは誰？
stop:How, jp:メーン州にはどのくらい住んでいるの？
stop:What, jp:どういう意味だい、それ？
stop:What, jp:何のために？
stop:What, jp:エミ叔母ちゃんは何時に来る予定なの？
stop:How, jp:いろはへ、元気にしていますか？
stop:Who, jp:だれが歌っているのかな？
stop:Why, jp:なんで泣いてるの？
stop:What, jp:何しているの？
stop:Who, jp:だれが指揮をしたいですか？
stop:Who, jp:だれがチキンアドボをつくれる？
stop:Who, jp:だれがきみのごはんをつくるの？
stop:Who, jp:あまねはだれを好きなの？
stop:How, jp:美術館への行き方を教えていただけますか？
stop:Where,

In [85]:
crawler.make_corpus()
crawler.write(data_path+corpus_name)

In [77]:
label_list = "YN WH please plain".split()
crawler = Crawler()
crawler.connect("192.168.1.224")
for mode in label_list:
    corpus_name = "{0}.json".format(mode)
    crawler.crawl(mode, 5)
    crawler.make_corpus()
    crawler.write(data_path+corpus_name)
crawler.quit()


stop:Can, jp:わたしたちを案内してくれませんか？
stop:Can, jp:写真をとってもいいですか？
stop:Do, jp:いっしょに行かない？
stop:Do, jp:心は本屋さんの近くに住んでいるの？
stop:Can, jp:ひとつ聞いてもいい？
stop:Did, jp:楽しかった？


WebDriverException: Message: Session [a88e5e83a0bd61fec860a29f334c5714] was terminated due to CLIENT_STOPPED_SESSION
Stacktrace:
    at org.openqa.grid.internal.ActiveTestSessions.getExistingSession (ActiveTestSessions.java:115)
    at org.openqa.grid.internal.DefaultGridRegistry.getExistingSession (DefaultGridRegistry.java:387)
    at org.openqa.grid.web.servlet.handler.RequestHandler.getSession (RequestHandler.java:241)
    at org.openqa.grid.web.servlet.handler.RequestHandler.process (RequestHandler.java:123)
    at org.openqa.grid.web.servlet.DriverServlet.process (DriverServlet.java:85)
    at org.openqa.grid.web.servlet.DriverServlet.doPost (DriverServlet.java:69)
    at javax.servlet.http.HttpServlet.service (HttpServlet.java:707)
    at javax.servlet.http.HttpServlet.service (HttpServlet.java:790)
    at org.seleniumhq.jetty9.servlet.ServletHolder.handle (ServletHolder.java:865)
    at org.seleniumhq.jetty9.servlet.ServletHandler.doHandle (ServletHandler.java:535)
    at org.se